In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle

#IMPORTING H5PY
###############################################################
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import h5py #also in sbatch add: export HDF5_USE_FILE_LOCKING=FALSE
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# # dx = 250m; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
# res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
##############################################
#DATA INITIALIZATION and LOADING FUNCTIONS

In [ ]:
#INITIALIZE DATA FUNCTION
###############################################################
def initiate_array(out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel1['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g', 'A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
#INITIALIZE DATA FUNCTION
def InitiateArray_Job(data,out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel1['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g', 'A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(data1['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust
# job_id=1;num_jobs=60
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# data=data1.isel(time=slice(start_job,end_job))
# parcel=parcel1.isel(time=slice(start_job,end_job))

In [ ]:
#################################################################################################################################################

In [ ]:
###############################################################
#FUNCTIONS

In [ ]:
# def GetZYX(parcel): #OLD #NO NEED TO RECALCULATE, ALREADY CALCULATED IN Lagrangian_Binary_Array
#     #Lagrangian Position Arrays
#     ##############
#     def grid_location(z,y,x):
#         zf=data1['zf'].values*1000; which_zh=np.clip(np.searchsorted(zf,z)-1,0,None).astype(np.uint16)
#         #which_zh=np.where(which_zh == -1, 0, which_zh) 
        
#         yf=data1['yf'].values*1000; which_yh=np.clip(np.searchsorted(yf,y)-1,0,None).astype(np.uint16) 
#         #which_yh=np.where(which_yh == -1, 0, which_yh) 
        
#         xf=data1['xf'].values*1000; which_xh=np.clip(np.searchsorted(xf,x)-1,0,None).astype(np.uint16)
#         #which_xh=np.where(which_xh == -1, 0, which_xh) 
        
#         return which_zh,which_yh,which_xh
    
#     print('Creating Lagrangian X,Y,Z Binary Arrays')
#     x=parcel['x'].data;y=parcel['y'].data;z=parcel['z'].data
#     Z,Y,X=grid_location(z,y,x)
    
#     return Z,Y,X

# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type,start_job,end_job):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
           data_dict = {var_name: f[var_name][start_job:end_job] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        data_dict = {k: in_data[k][start_job:end_job].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

def GetZYX(start_job,end_job):
    import h5py
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['Z', 'Y', 'X']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    Z, Y, X = (data_dict[k] for k in var_names)
    return Z,Y,X
#[Z,Y,X] GetZYX(parcel,start_job,end_job)

In [ ]:
#constants
Cp=1004 #Jkg-1K-1
Cv=717 #Jkg-1K-1
Rd=Cp-Cv #Jkg-1K-1
eps=0.608

Lx=(data1['xf'][-1].item()-data1['xf'][0].item())*1000 #x length (m)
Ly=(data1['yf'][-1].item()-data1['yf'][0].item())*1000 #y length (m)
Np=len(parcel1['xh']) #number of lagrangian parcles
dt=(data1['time'][1]-data1['time'][0]).item()/1e9 #sec
dx=(data1['xf'][1].item()-data1['xf'][0].item())*1e3 #meters
dy=(data1['yf'][1].item()-data1['yf'][0].item())*1e3 #meters
xs=data1['xf'].values*1000
ys=data1['yf'].values*1000
zs=data1['zf'].values*1000

def zf(z):
    k=z #z is the # level of z
    out=data1['zf'].values[k]*1000
    
    return out
# def rho(x,y,z,t):
#     p=data1['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
#     p0=101325 #Pa
#     theta=data1['th'].isel(xh=x,yh=y,zh=z,time=t).item()
#     T=theta*(p/p0)**(Rd/Cp)
#     qv=data1['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
#     # Tv=T*(1+eps*qv)
#     Tv=T*(eps+qv)/(eps*(1+qv))
#     rho = p/(Rd*Tv)
#     out=rho
#     return out

def rho(x,y,z,rho_data_t):
    out=rho_data_t[z,y,x]
    return out
def m(t):
    rho_data_t=data1['rho'].isel(time=t).data
    
    m=0
    #triple sum
    for k in range(len(data1['zh'])):
        dz=(zf(k+1)-zf(k))
        for j in range(len(data1['yh'])):
            for i in range(len(data1['xh'])):
                rho_out=rho(i,j,k,rho_data_t)
                m+=rho_out*dz
                
    #triple sum
    out=m*dx*dy/Np
    return out


In [ ]:
#Calculate Mass Constant
# calculate='single_time'
# calculate=True
calculate=False

if calculate==True:
    Nt=len(data1['time'])
    m_arr=np.zeros((Nt))
    for t in np.arange(Nt):
        if np.mod(t,25)==0: print(t)
        m_arr[t]=m(t)
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
    np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
elif calculate=='single_time':
    Nt=len(data1['time'])
    m_arr=np.zeros((Nt))

    t=0 #len(data1['time'])//2 #Pick some middle time
    m_300=m(t)
    for t in np.arange(Nt):
        m_arr[t]=m_300 #UNCOMMENT FOR FULL CALCULATION
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
    np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
else:
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
    m_arr = np.load(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy')

# # TESTING
# lst=[]
# for t in np.arange(133):
#     lst.append(m_arr[t])

# plt.plot(lst)
# (np.max(lst)-np.min(lst))*100/np.mean(lst)

In [ ]:
PROCESSING=False
PROCESSING=True

In [ ]:
def call_variable(data,varname):
    #READING BACK IN

    
    # print('loading vars')
    if PROCESSING==False:
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
    if PROCESSING==True:
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(dir3, "r") as h5f:
        if varname=='e_g':
            profile_array = h5f["profile_array_e_g"][start_job:end_job]
        elif varname=='e_c':
            profile_array = h5f["profile_array_e_c"][start_job:end_job]
        elif varname=='d_g':
            profile_array = h5f["profile_array_d_g"][start_job:end_job]
        elif varname=='d_c':
            profile_array = h5f["profile_array_d_c"][start_job:end_job]
    
    def apply_constant(profile_array,apply):
        if apply==True:
            Nt=profile_array.shape[0]
            Nz=profile_array.shape[1]
        
            profile_array/=(dx*dy*dt)
            for t in np.arange(Nt):
                profile_array[t]*=m_arr[t+index_adjust]
            for z in np.arange(Nz):
                dz=zf(z+1)-zf(z)
                profile_array[:,z]/=dz
        return profile_array
    
    #APPLY CONSTANTS TO ENTRAINMENT VALUE
    ##################################################
    
    profile_array=apply_constant(profile_array,apply=True)
    if varname in ['d_g','d_c']:
        profile_array*=-1
    ##################################################

    return profile_array

In [ ]:
# def MakeLagrangianArray(data,varnames,Z,Y,X): #SLOW FOR LOOP VERSION
#     print('Making Variables Lagrangian Array')
#     # Initialize dictionaries
#     var_data_dict = {varname: call_variable(data,varname) for varname in varnames}
#     VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

#     Nt = len(data['time'])
#     Np = len(parcel1['xh'])
#     for p in np.arange(Np):
#         # if np.mod(p, 1e6) == 0: 
#         #     print(f"{p}/{len(parcel1['xh'])}")

#         # Get Indices
#         zs = Z[:, p]
#         ys = Y[:, p]
#         xs = X[:, p]
#         ts = np.arange(Nt)

#         # Loop over all variables and fill the respective VAR array
#         for varname, var_data in var_data_dict.items():
#             VAR[varname][:, p] = var_data[ts, zs, ys, xs]

#     # return [VAR[varname] for varname in varnames]
#     return VAR

def MakeLagrangianArray(data,varnames,Z,Y,X): #FAST ADVANCED INDEXING VERSION
    print('Making Variables Lagrangian Array')
    # Initialize dictionaries
    var_data_dict = {varname: call_variable(data,varname) for varname in varnames}
    VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

    print('Running')
    Nt = len(data['time'])
    # Get Indices
    z_idx = Z
    y_idx = Y
    x_idx = X
    ts = np.arange(Nt)[:, None]
    t_idx = np.broadcast_to(ts, Z.shape)

    for varname, var_data in var_data_dict.items():
        VAR[varname] = var_data[t_idx, z_idx, y_idx, x_idx]

    # return [VAR[varname] for varname in varnames]
    return VAR

In [ ]:
def SaveData(data,varnames,VAR):
    # Saving Data
    ##############
    print('Saving Data\n')
    import h5py
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/SBATCH/job_out/'
    if PROCESSING==True:
        out_file=dir2+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}'
    elif PROCESSING==False:
        out_file=dir2+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}_ORIGINAL'
    out_file+=f'_{job_id}.h5'

    out_varnames=[varname.upper() for varname in varnames]
    InitiateArray_Job(data,out_file,out_varnames,t_chunk_size=1,p_chunk_size=1_000_000)
    with h5py.File(out_file, 'a') as f: 
        for var in VAR:
            f[var.upper()][:] = VAR[var]
# SaveData(data,varnames,VAR)            

In [ ]:
################################
#RUNNING

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if res == "1km" and Np_str=='1e6':
    num_jobs=60 #1M parcels
    num_slurm_jobs=10
elif res == "1km" and Np_str=='50e6':
    num_jobs=400 #50M parcels
    num_slurm_jobs=60
elif res == "250m" and Np_str=='50e6':
    num_jobs=600 #50M parcels
    num_slurm_jobs=150
##############################

In [ ]:
################################
#RUN SETUP
###############################
varnames=['e_c','d_c','e_g','d_g']

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array
print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}/{num_jobs}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

    #SLICING DATA
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
    data=data1.isel(time=slice(start_job,end_job))
    parcel=parcel1.isel(time=slice(start_job,end_job))

    #GETTING POSITION VARIABLES
    [Z,Y,X]=GetZYX(start_job,end_job)

    #MAKING LAGRANGIAN ARRAYS (for threshold)
    VAR=MakeLagrangianArray(data,varnames,Z,Y,X)

    #SAVING
    SaveData(data,varnames,VAR)

    #check_memory(globals())
    del VAR

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
recombine=True

In [ ]:
# #OLD
# def Recombine(num_jobs):
#     dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
#     dir3=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
#     out_file=dir3+f'lagrangian_VAR_array_{res}_{t_res}_{Np_str}.h5'

#     print('initializing array')
#     vars=['u','w','z','Z','Y','X','W','QCQI','A_g','A_c']
#     InitiateArray_Job(out_file,vars,t_chunk_size=100,p_chunk_size=500_000)

#     print('recombining')
#     with h5py.File(out_file, 'r+') as f_out:
#         for job_id in np.arange(1,num_jobs+1):
#             if job_id % 5: print(f"job_id = {job_id}")
#             [a,b] = get_job_range(job_id,num_jobs)
    
#             in_file=dir2+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}_{job_id}.h5' 
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][a:b]=f_in[var][:]

In [ ]:
def Recombine_Dask():
    #DASK-ENABLED
    import glob
    import re
    def recombine_func(in_files,out_file):
        # matching_files = sorted(glob.glob(in_files))
        matching_files = sorted(
        glob.glob(in_files),
        key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
    )
        
        print(f'recombining {len(matching_files)} files')
        out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_0',combine='nested',phony_dims='sort')
        from dask.diagnostics import ProgressBar
        with ProgressBar():
            out.to_netcdf(out_file, engine='h5netcdf')
        
    if recombine==True:
        dir2=dir+'Project_Algorithms/Lagrangian_Arrays/SBATCH/job_out/'
        dir3=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
        if PROCESSING==True:
            in_files = dir2 + f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}_[0-9]*.h5'
            out_file=dir3+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}.h5' 
        elif PROCESSING==False:
            in_files = dir2 + f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}_ORIGINAL_*.h5'
            out_file=dir3+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}_ORIGINAL.h5' 
        
        recombine_func(in_files,out_file)

In [ ]:
if recombine==True:
    # Recombine(num_jobs=num_jobs)
    Recombine_Dask()

In [ ]:
##########################################################################################

In [ ]:
# #LOADING BACK IN
# #TESTING
# start_job=int(500/5);end_job=int(505/5)
# # start_job=500;end_job=502
# data2=data.isel(time=slice(start_job,end_job))

# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# out_file=dir2+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(open_file, 'r') as f:
#     # Load the dataset by its name
#     VAR = f['VAR'][start_job:end_job]
#     # for key in f:
#         # globals()[key]=f[key][start_job:end_job]
# check_memory(globals())

In [ ]:
########################
# TESTING

In [ ]:
# #TESTING (IS VERSION3==VERSION2)
# job_id=3
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# # start_job,end_job=0,3

# #TESTING
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# open_file=dir2+f'VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(open_file, 'r') as f:
#     # Load the dataset by its name
#     for key in f:
#         globals()[key+"1"]=f[key][start_job:end_job]
#     # QV = f['QV'][start_job:end_job]
#     # TH = f['TH'][start_job:end_job]
#     # TH_E = f['TH_E'][start_job:end_job]
#     # BUOYANCY = f['BUOYANCY'][start_job:end_job]
#     # HMC = f['HMC'][start_job:end_job]
#     # VMF_c = f['VMF_c'][start_job:end_job]
# check_memory(globals())

# job_id=3
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
# out_file=dir2+f'lagrangian_VARS_array_{res}_{t_res}_{Np_str}'
# out_file+=f'_{job_id}.h5'
# with h5py.File(out_file, 'r') as f:
#     # Load the dataset by its name
#     for key in f:
#         globals()[key+"2"]=f[key][:]
# check_memory(globals())

# print(np.all(QV1==QV2))
# print(np.all(TH1==TH2))
# print(np.all(TH_E1==TH_E2))
# print(np.all(BUOYANCY1==BUOYANCY2))
# print(np.all(HMC1==HMC2))
# print(np.all(VMF_c1==VMF_C2))
# print(np.all(VMF_g1==VMF_G2))

In [ ]:
# #TESTING ARE RESULTS CORRECT
# job_id=50;num_jobs=60
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# # start_job,end_job=0,3

# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# with h5py.File(dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5', 'r') as f:
#     # Load the dataset by its name
#     Z = f['Z'][start_job:end_job]
#     Y = f['Y'][start_job:end_job]
#     X = f['X'][start_job:end_job]

# #TESTING

# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
# open_file=dir2 + f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}_{job_id}.h5'
# with h5py.File(open_file, 'r') as f:
#     key_list=list(f.keys())
#     for key in f:
#         print(key)
#         globals()[key]=f[key][:]
# data2=data1.isel(time=slice(start_job,end_job))

In [ ]:
# #TESTING ARE RESULTS CORRECT
# t,p=0,32
# def test(t,p,VAR,var):
#     z=Z[t,p];y=Y[t,p];x=X[t,p]
#     data3=call_variable(data2,var)
#     out=data3[t,z,y,x]
#     print(VAR[t,p],out)
# for key in key_list:
#     test(t,p,globals()[key],key.lower())
# # np.where(globals()[key]!=0)